<div style="page-break-before: always;"/> 

# Polarization

M&I 4e

Ch 14 Electric Fields and Matter

- §14.3 &ndash; Polarization of Atoms
- §14.4 &ndash; Polarization of Insulators
- §14.6 &ndash; Charge Motion in Metals

Ch 15 Electric Field of Distributed Charges

- §15.1 &ndash; A Uniformly Charged Thin Rod
- §15.2 &ndash; Procedure for Calculating Electric Field

In [ ]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]*
>
>   There is a region where an electric field points to the right, due to charged
>   particles somewhere. 
>
>   A neutral carbon atom is placed inside this region. 
>
>   Draw a diagram of the situation. __Which of the following statements
>   are correct?__
>
>   (check all that apply:)
>
>   - [ ] &nbsp; The neutral carbon atom polarizes, and becomes a dipole.
>   
>   - [ ] &nbsp; The electron cloud in the carbon atom shifts to the left.
>
>   - [ ] &nbsp; The nucleus of the carbon atom shifts to the left.
>
>   - [ ] &nbsp; Because the net charge of the carbon atom is zero, 
>                it can not be affected by an electric field. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

- [x] &nbsp; The neutral carbon atom polarizes, and becomes a dipole.

- [x] &nbsp; The electron cloud in the carbon atom shifts to the left.

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]*
>
>   {{ graphic }}
>
>   A charged particle with charge $q_1$ is a distance $r$ from a neutral atom,
>   as shown in the diagram. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='2a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(A)
>
>   If $q_1$ is negative, __which diagram (below) best shows the charge 
>   distribution of the neutral atom__ in this situation?
>
>   {{ graphic }}
>
>   (select one:)
>
>   - [ ] &nbsp; 2
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] 2

<h3 id='2b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the direction of the electric field at the location of the charged
>   particle__, made by the polarized neutal atom?
>
>   {{ graphic }}
>
>   (select one:)
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] g

<h3 id='2c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the direction of the force on the charged particle, due to the 
>   polarized neutral atom?__
>
>   {{ graphic }}
>
>   (select one:)
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] c


<h3 id='2d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the direction of the force on the polarized neutral atom due to 
>   the charged particle?__
>
>   {{ graphic }}
>
>   (select one:)
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] g

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]
>
>   A charged tape is brought near your hand, as shown in the diagram above.
>   Your hand is initially electrically neutral. 
>
>   {{ graphic }}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='3a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(A)
>
>   If the tape is negatively charged, __which diagram (below) best shows the 
>   polarization of a neutral molecule in your hand?__
>
>   {{ graphic }}
>
>   (select one:)
>
>   - [ ] &nbsp; 2
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] 2

<h3 id='3b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the direction of the electric field at the location of the tape 
>   due to the large number of polarized molecules in your hand?__
>
>   {{ graphic }}
>
>   (select one:)
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] g

<h3 id='3c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the direction of the force on the tape, due to the 
>   polarized molecules in your hand?__
>
>   {{ graphic }}
>
>   (select one:)
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] c


<h3 id='3d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the direction of the force on your hand, due to the charged tape?__
>
>   {{ graphic }}
>
>   (select one:)
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] g

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]
>
>   Try rubbing a plastic pen through your hair, and you'll find that you can 
>   pick up a tiny scrap of paper, when the pen is about one centimeter ([cm])
>   above the paper. From this simple experiment, you can estimate how much an 
>   atom is polarized by the pen! You will need to make several assumptions and
>   approximations.
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(A)
>
>   Suppose that the center of the outer electron cloud ($q = -4e$) of a carbon
>   atom shifts a distance $s$ when the atom is polarized by the pen. 
>
>   __Calculate $s$ algebraically, in terms of the charge $Q$ on the pen.__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $s = \fbox{\hspace{8em}?}$
>
>   (Use the following as necessary: $m$, $g$ for acceleration due to gravity,
>    $h$ for the distance from the carbon to the pen, $e$ for the charge of the
>    electron, $Q$, $\epsilon_0$, and $\pi$.)
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='4b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(B)
>
>   Assume that the pen carries $Q = 10^{-8}$ [C] of charge.
>
>   __Evaluate $s$ numerically.__ Enter your answer with 3 significant digits. 
>
>   How does this compare with the size of an atom or a nucleus?
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $s = \fbox{\hspace{8em}?} \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='4c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(C)
>
>   __Calculate the polarizability $\alpha$ of a carbon atom.__
>   Enter your answer with 3 significant digits. 
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\alpha = \fbox{\hspace{8em}?} \left[\mathrm{m^3 / (N⋅m^2) / C^2}\right]$
>
>   Compare your answer to the measured value of 
>   $1.96 \times 10^{-40} \left[\mathrm{C ⋅ m}\ / (\mathrm{N / C}) \right]$
>
>   (T.M. Miller and B. Bederson, "Atomic and molecular polarizabilities: a 
>   review of recent advances", _Advances in Atomic and Molecular Physics_ 
>   13, 1-55, 1977).
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]</h2>

>   ---
>   ## [5]
>
>   A solid plastic ball has negative charge, uniformly spread over its surface.
>
>   __Which of the figures below best shows the polarization of molecules inside the ball?__
>
>   - [ ] (a) {{ graphic }}
>
>   - [ ] (b) {{ graphic }}
>
>   - [ ] (c) {{ graphic }}
>
>   - [ ] (d) {{ graphic }}
>
>   - [ ] None of these apply. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [6]
>
>   A negatively charged iron block is placed in a region, where there is an 
>   electric field downward (in the $-y$ direction), due to charges not shown. 
>
>   __Which of the diagrams best describes the charge distribution in, and/or on,
>   the iron block?__
>
>   - [ ] (a) {{ graphic }}
>
>   - [ ] (b) {{ graphic }}
>
>   - [ ] (c) {{ graphic }}
>
>   - [ ] (d) {{ graphic }}
>
>   - [ ] (e) {{ graphic }}
>
>   - [ ] (f) {{ graphic }}
>
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]</h2>

>   ---
>   ## [7]*
>
>   Two small, negatively charged, plastic spheres, are placed near a neutral 
>   iron block, as shown in the figure below. 
>
>   __Which arrow ($a$-$j$) best indicates the direction of the net electric 
>   field at location $A$ in the diagram below?__
>
>   {{ graphic }}
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

- [x] f

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   ---
>   ## [8]
>
>   __Which of the following diagrams correctly displays the polarization of a 
>   metal sphere, by an electric field that points downward__; using the 
>   conventions discussed in the _Matter and Interactions__ textbook?
>
>   {{ graphic a | graphic b | graphic c | graphic d }}\
>   {{ graphic e | graphic f | graphic g | graphic h }}\
>   {{ graphic j | graphic k | graphic l | graphic m }}
>
>   |   |   |   |   |
>   |:---:|:---:|:---:|:---:|
>   | 🔲 a | 🔲 b | 🔲 c | 🔲 d |
>   | 🔲 e | 🔲 f | 🔲 g | 🔲 h |
>   | 🔲 j | 🔲 k | 🔲 l | 🔲 m |
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='9' style="display:none">[9]</h2>

>   ---
>   ## [9]
>
>   A plastic rod of $1.6$ [m] is rubbed all over with wool, and acquires a charge
>   of $-8 \times 10^8$ coulombs ([C]).
>
>   We choose the center of the rod to be the origin of our coordinate system; 
>   with the $x$-axis extending to the right, $y$-axis extending up the page,
>   and the $z$-axis out of the page. 
>
>   In order to calculate the electric field at location $A = \left\langle\ 0.7, 0, 0\ \right\rangle\ \left[\mathrm{m}\right]$,
>   we divide the rod into 8 pieces, and approximate each piece as a point charge, located at the center of the piece. 
>
>   {{ graphic }}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='9a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the length of one of these pieces?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the location of the center of piece number 2?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(C)
>
>   __How much charge is on piece number 2?__
>   (Remember that the charge is negative.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{coulombs}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(D)
>
>   Approximating piece 2 as a point charge, __what is the electric field at location $A$, due 
>   only to piece 2?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_2 = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(E)
>
>   To get the net electric field at location $A$, we would need to calculate 
>   $\vec{E}$ due to each of the 8 pieces, and add up these contributions.
>
>   If we did that, __which direction arrow above would best represent the 
>   direction of the net electric field, at location $A$?__
>
>   (select one:)
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='10' style="display:none">[10]</h2>

>   ---
>   ## [10]*
>
>   An electrostatic dust precipitator that is installed in a factory smokestack
>   includes a straight metal wire of length $L = 0.8$ [m], that is charged with
>   approximately uniformly with a total charge $Q = 0.2 \times 10^{-7}$ [C].
>
>   A speck of coal dust (which is mostly carbon) is near the wire, and far from
>   both ends of the wire; the distance from the wire to the speck is $d = 1.2$ 
>   [cm]. Carbon has an atomic mass of 12 (6 protons and 6 neutrons in the 
>   nucleus). A careful measurement of the polarizability of a carbon atom gives
>   the value $\alpha = 1.96 \times 10^{-40}$ [(C⋅m)/(N/C)]. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='10a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Calculate the initial acceleration of the speck of coal dust, 
>   neglecting gravity.__
>
>   Your answer must be expressed in terms of $Q$, $L$, $d$, and $\alpha$.
>   You can use other quantities in your calculations, but your final result
>   must not include them. 
>
>   It is convenient to use the "binomial expansion", that you may have learned
>   in calculus, that $(1 + \epsilon)^n \approx 1 + n\epsilon$, if $\epsilon \ll 1$.
>   Note that $n$ can be negative. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h4 id='10a_i' style="display:none">{A.i}</h4>

>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>
>   
>   #### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;{A.i}
>
>   __Algebraically express the magnitude of the acceleration $a$.__
>
>   Use the following as necessary: $Q$, $L$, $d$, $\alpha$, $\epsilon_0$, and
>   $m$ for the mass of a carbon atom. Use the binomial expansion in your answer.
>
>   (select one:)
>
>   - [ ] &nbsp; $a = \left(\frac{1}{4\pi\epsilon_0}\right)^2 \frac{4 Q^2 \alpha}{m L^3 d^2}$
>
>   - [ ] &nbsp; $a = \left(\frac{1}{4\pi\epsilon_0}\right)^2 \frac{Q^2 \alpha}{m L^2 d^3}$
>
>   - [ ] &nbsp; $a = \left(\frac{1}{4\pi\epsilon_0}\right)^2 \frac{4 Q^2 \alpha}{m L^2 d^3}$
>
>   - [ ] &nbsp; $a = \left(\frac{1}{4\pi\epsilon_0}\right)^2 \frac{Q^2 \alpha}{m L^3 d^2}$
>
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<h4 id='10a_ii' style="display:none">{A.ii}</h4>

>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>
>   
>   #### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;{A.ii}
>
>   __Calculate the initial acceleration of the speck of coal dust, neglecting
>   gravity.__
>
>   Don't put numbers into your calculation until the very end, but then show
>   the numerical calculation that you carry out on your calculator.
>
>   (Enter the magnitude:)
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{m / s^2}\right]$
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<h3 id='10b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(B)
>
>   If the speck of coal dust were initially six times as far from the charged
>   wire, __how much smaller would the initial acceleration of the speck be?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\frac{a_{6d}}{a_d} = \fbox{\hspace{8em}?}$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='11' style="display:none">[11]</h2>

>   ---
>   ## [11]
>
>   A strip of invisible tape, $0.15$ [m] long by $0.013$ [m] wide, is uniformly
>   charged with a total net charge of $2$ [nC] (nano `n` = $1 \times 10^{-9}$);
>   and is suspended horizontally so it lies along the $x$-axis, with its center
>   at the origin (as shown in the figure below). 
>
>   {{ graphic }}
>
>   Calculate the approximate electric field at location A, 
>   $\left\langle\ 0, 0.07, 0\right\rangle$ [m], due to the strip of tape. 
>
>   Do this by dividing the strip into three equal sections, as in the figure 
>   above, and approximating each section as a point charge. (Assume each point
>   charge is located at the center of the section it approximates.)
>
>   Express your answers in vector form. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='11a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B9;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the approximate electric field at $A$, due to piece 1?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_1 =  \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='11b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B9;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the approximate electric field at $A$, due to piece 2?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_2 =  \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='11c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B9;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the approximate electric field at $A$, due to piece 3?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_3 =  \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='11d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B9;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the approximate _net_ electric field at $A$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_3 =  \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='11e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B9;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What could you do to improve the accuracy of your calculations?__
>
>   (select one:)
>
>   - [ ] &nbsp; Use a _smaller_ charge for piece 2, than for pieces 1 and 3:
>                the actual electric field is similar to that of a dipole, 
>                with most of the charge on the tape at the left or right ends.
>
>   - [ ] &nbsp; Use a _larger_ charge for piece 2, than for pieces 1 and 3:
>                since location $A$ is closest to piece 2, this piece has the 
>                most significant contribution. 
>
>   - [ ] &nbsp; Use more sections: as the number of segments approaches infinity,
>                the summation of the electric fields approximates the actual 
>                electric field. 
>
>   - [ ] &nbsp; Use just two sections: due to symmetry about the $y$-axis, 
>                the components of the field due to each charge will be half of
>                the total electric field. 
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

---